### Introduction

Welcome to Priceless Wheels! In this project, our goal is to build a model that can accurately predict the price of a used vehicle based on various factors such as make, model, year, mileage, and condition. The automobile industry is one of the largest and most competitive industries in the world, with millions of vehicles being sold each year. The price of a vehicle can have a significant impact on a consumer's purchasing decision and it is important for both buyers and sellers to have an understanding of the market value of a vehicle. By using machine learning algorithms and data analysis, we aim to provide a reliable and robust model that can assist in determining the fair market value of a vehicle. Join us on this exciting journey as we delve into the world of vehicle price prediction.

### About the data

This data is scrapped from https://www.cardekho.com/. This data is meant for research and academic purposes only and is **not meant for commercial use**. This dataset contains about 38000 (thirty eight thousand) used cars listed in CarDekho in India. Download the data from https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv.

---

#### Importing the data and libraries

Let's start by importing the necessary libraries and the data.

In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import regex as re # for regex matching
import matplotlib.pyplot as plt # 
import seaborn as sns # seaborn for nice looking plots
sns.set() # setting seaborn default for plots

# removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Next, we will import the data and preview the first 5 rows

- Note: The ```usedCarSkuId``` is a unique identifier for each car. We would be using this column as the index of the dataframe
- Note: Since we are using github lfs for storing the CSV files, the link mentioned in the notebook will expire in some time. Please visit this [link](https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv) to get the data

In [25]:
# The data can be found on kaggle link: https://www.kaggle.com/datasets/sukritchatterjee/used-cars-dataset-cardekho

# file_path = "/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv"
file_path = '../data/raw/cardekho_cars_2023_03_19_16_44_14.csv'
df = pd.read_csv(file_path, index_col="usedCarSkuId")

# sanity check
df.head()

/var/folders/vt/5_mmhf_d5gd9_w9qt3c4rvlw0000gn/T/ipykernel_33937/199014853.py:5: DtypeWarning: Columns (25,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, index_col="usedCarSkuId")


,position,loc,myear,bt,tt,ft,km,ip,pi,images,...,owner_type,price_segment_new,template_name_new,page_template,template_Type_new,experiment,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,1,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,2,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,3,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,4,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,NaN,[{'img': ''}],...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,5,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,10+lakh,used cardetail v2,Used Car > Detail Page,used,control,NaN,12.0+-.03,NaN,NaN


The data has imported with some mixed data types. We will need to convert the data types as we explore the data.

In [26]:
# checking the shape of the dataset
shape = df.shape

print(f"The shape of the dataframe is {shape[0]} rows and {shape[1]} columns")

The shape of the dataframe is 37814 rows and 139 columns


As we can see, there are 37,814 rows and 139 columns in the dataset.

Let's look at the data types of the columns.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37814 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Columns: 139 entries, position to Ground Clearance Unladen
dtypes: bool(3), float64(10), int64(28), object(98)
memory usage: 39.6+ MB


Let's take a look at the feature dictionary provided with the dataset.

In [28]:
# feature_dictionary = pd.read_csv("/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv")
feature_dictionary = pd.read_csv("../data/feature_dictionary_raw.csv")
feature_dictionary

Now comes the difficult task. Go through the feature dictionary and remove the unwanted columns - features which might be repeated but not totally apparent. Note down the columns which makes sense to keep and are not repeating and delete the rest.

In [29]:
columns_to_keep = [
    "loc",
    "myear",
    "bt",
    "tt",
    "ft",
    "km",
    "ip",
    "images",
    "imgCount",
    "threesixty",
    "dvn",
    "oem",
    "model",
    "variantName",
    "city_x",
    "pu",
    "discountValue",
    "utype",
    "carType", 
    "top_features",
    "comfort_features",
    "interior_features",
    "exterior_features",
    "safety_features",
    "Color",
    "Engine Type",
    "Max Power",
    "Max Torque",
    "No of Cylinder",
    "Values per Cylinder",
    "Value Configuration",
    "BoreX Stroke",
    "Turbo Charger",
    "Super Charger",
    "Length",
    "Width",
    "Height",
    "Wheel Base",
    "Front Tread",
    "Rear Tread",
    "Kerb Weight",
    "Gross Weight",
    "Gear Box",
    "Drive Type",
    "Seating Capacity",
    "Steering Type",
    "Turning Radius",
    "Front Brake Type",
    "Rear Brake Type",
    "Top Speed",
    "Acceleration",
    "Tyre Type",
    "No Door Numbers",
    "Cargo Volumn",
    "model_type_new",
    "state",
    "owner_type",
    "exterior_color",
    "Fuel Suppy System",
    "Compression Ratio",
    "Alloy Wheel Size",
    "Ground Clearance Unladen",
]

df.drop([x for x in df.columns if x not in columns_to_keep], axis=1, inplace=True)
print(f"After dropping some unnecessary columns, the dataset now has {df.shape[1]} columns. These columns are hand picked and will be be further analyzed.")

After dropping some unnecessary columns, the dataset now has 62 columns. These columns are hand picked and will be be further analyzed.


---

### Data cleaning and wrangling

* Dropping duplicate rows
* Fixing the values and data types of the columns
* Checking for multicollinearity and determining how to handle it
* Dropping irrelevant columns for the model
* Saving the cleaned data for the next part of the project

---

### Duplicates

We know that there should be no duplicates in the data. They are checked for and removed at the time of data collection. *(Trust me, I collected it myself 😉 )*

In [30]:
# checking for duplicate rows
duplucate_rows = df.duplicated().sum()

print(f"The number of duplicate rows are {duplucate_rows}.")

The number of duplicate rows are 1.


Somehow, one duplicate row sneaked in (that's awkward 🫣)! Let's check the rows.

In [31]:
# setting the duplicated index
duplicate_index = df.duplicated(keep=False)

# calling the duplicated index in a dataframe
df.loc[duplicate_index, :].sort_index()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
aa39e640-6183-4379-a517-9f5b2458b2a5,NaN,2014,Hatchback,Manual,Diesel,"70,000",0,[{'img': ''}],0,False,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN
ce8b30d8-c438-4a2f-bce1-5c1887a95495,NaN,2014,Hatchback,Manual,Diesel,"70,000",0,[{'img': ''}],0,False,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN


As we can see, the rows are exactly the same. This means we can safely drop the row. Since the we indexed by the usedCarSkuId, we need to drop one of them by their index.

In [32]:
# making a copy with the duplicated rows dropped
df2 = df.drop_duplicates().copy()

# checking for duplicate rows in the new dataframe
dup = df2.duplicated().sum()

print(f"The number of duplicate rows are {dup}.")

The number of duplicate rows are 0.


In [33]:
# sanity check
df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,180-liters,used,uttar pradesh,Silver,first,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,235-litres,used,maharashtra,Grey,first,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,4.000,400-litres,used,delhi,Silver,second,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,[{'img': ''}],0,False,...,4.000,NaN,used,delhi,Silver,second,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,5.000,NaN,used,maharashtra,White,first,NaN,12.0+-.03,NaN,NaN


Great! Now that we have removed the duplicate rows, let's look at missing values

---

### Fixing the data types

Let's look at the data types of the columns and in the process we can check if we need to clean any them to make them more useful, such as converting them to a numeric type.

*When we do feature engineering we will further modify these columns and might also drop a few of them*

We already know about the current data types of the features from the ```feature_dictionary```.

In [34]:
# Filter the dataframe to only include the columns_to_keep
new_feature_dict = feature_dictionary.copy()

# Drop the rows which Feature is not in the columns_to_keep list
row_index = [i for i, x in enumerate(new_feature_dict['Feature']) if x not in columns_to_keep]
new_feature_dict.drop(row_index, inplace=True)

new_feature_dict

,Feature,Type,Sample,Missing Values,Unique Values,Description
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
5,ft,object,CNG,0,5,Fuel type of the car
...,...,...,...,...,...,...
130,owner_type,object,first,0,6,"Owner type of the car (first, second, etc.)"
136,Fuel Suppy System,object,NaN,5502,99,"Type of fuel supply system (Carburetor, Fuel I..."
137,Compression Ratio,object,NaN,27642,100,Compression ratio of the engine
138,Alloy Wheel Size,object,NaN,13146,18,Size of the alloy wheels in inches


We know that if a column has the ```object``` type, it means it could have mixed types as well as strings. Let's try to convert them to a numerical type wherver we encouter something like Length or '12mm'.

In [35]:
# A helper function to print all the types in a column for a dataframe
def print_samples_from_every_type(df: pd.DataFrame, col: str):
    # Get all the types in the column.
    types = df[col].apply(type).unique()
    for t in types:
      # Print 10 samples from each type.
        print(f"Samples for type {t}: ")
        print(df[df[col].apply(type) == t].sample(10)[col])
        print()

In [36]:
# Make a copy of the original dataframe so that we can roll back to it anytime.
cars_df = df2.copy()

---

Let's list down all the columns which need the standard string treatment: 
1. Convert column to be a string type
2. Strip the whitespace 
3. Make it lowercase

*Here we are also give some of the other columns (that we wish to convert later) the standard treatment (it will help us later)*

In [37]:
columns_str_to_lower = [
    'loc',
    'bt',
    'ft',
    'tt',
    'images',
    'dvn',
    'oem',
    'model',
    'variantName',
    'city_x',
    # 'pu': 3,70,000 -> 370000
    'utype',
    'carType',
    'top_features',
    'comfort_features',
    'interior_features',
    'exterior_features',
    'safety_features',
    'Color',
    'Engine Type',
    'Max Power', # Max Power : 33.54bhp@4000 rpm -> 2 columns 33.54, 4000
    'Max Torque', # Max Torque : 40.2Nm@3500 rpm -> 2 columns 40.2, 3500
    # No of Cylinder : float -> int,
    # Values per Cylinder : float -> int, !! Fix the name -> Valve per Cylinder
    'Value Configuration', # !! Fix the name -> Valve Configuration
    'BoreX Stroke', # BoreX Stroke : 69 x 72 mm -> Bore: 69, Stroke: 72
    'Turbo Charger', # Convert to boolean
    'Super Charger', # Convert to boolean
    'Length', # Length : 3599mm -> 3599
    'Width', # Width : 1495mm -> 1495
    'Height', # Height : 1700mm -> 1700
    'Wheel Base', # Wheel Base : 2400mm -> 2400
    'Front Tread', # Front Tread : 1295mm -> 1295
    'Rear Tread', # Rear Tread : 1295mm -> 1295
    'Kerb Weight', # Kerb Weight : 960kg -> 960
    'Gross Weight', # Gross Weight : 1350kg -> 1350
    'Gear Box', # might need some additional cleaning
    'Drive Type', # might need some additional cleaning
    # Seating Capacity : float -> int
    'Steering Type', # might need some additional cleaning
    'Turning Radius', # Turning Radius : 4.6 metres -> 4.6
    'Front Brake Type', # might need some additional cleaning
    'Rear Brake Type', # might need some additional cleaning
    'Top Speed', # Top Speed : 137 kmph -> 137
    'Acceleration', # Acceleration : 13.5 seconds -> 13.5
    'Tyre Type', # might need some additional cleaning
    # No Door Numbers : float -> int !! fix name
    'Cargo Volumn', # 'Cargo Volumn' : 300 litres -> 300, !! fix name
    'model_type_new', # !! fix name
    'state',
    'exterior_color',
    'owner_type', # might need some additional cleaning
    'Fuel Suppy System', # might need some additional cleaning
    # Compression Ratio : 10.0:1 -> 10.0
    'Alloy Wheel Size', # Alloy Wheel Size : convert to float
    'Ground Clearance Unladen', # Ground Clearance Unladen : 170mm -> 170
]

In [38]:
# Apply the changes
temp_arr = []

for col in columns_str_to_lower:
    cars_df[col] = cars_df[col].astype(str).str.strip().str.lower()
    
    # Print a nice sample
    while (True):
        found = 0
        vals = [v for v in cars_df[col].sample(100)]
        for val in vals:
            if val is not None:
                found = 1
                vals = val
                break
        
        if found == 1:
            temp_arr.append({ 
                "Feature" : col, 
                "Type": f'{cars_df[col].dtype}', 
                "Unique": cars_df[col].nunique(), 
                "Sample": vals
            })
            break

pd.DataFrame(temp_arr)

,Feature,Type,Unique,Sample
0,loc,object,398,new delhi g.p.o.
1,bt,object,12,suv
2,ft,object,5,petrol
3,tt,object,2,automatic
4,images,object,37135,[{'img': 'https://images10.gaadi.com/usedcar_i...
5,dvn,object,4128,hyundai accent executive cng
6,oem,object,46,hyundai
7,model,object,382,honda jazz
8,variantName,object,3430,xv
9,city_x,object,617,gurgaon


There is an additional issues here that needs to be addressed: 
Some of the columns have **more unique values** than are expected. For eg: `Turbo Charger` should either be a yes or a no or a NaN, beter are `5` unique values. The following are the columns suspected to have a higher than expected cardinality due to some quality issues: 
1. `Value Configuration`
2. `Turbo Charger`
3. `Gear Box`
4. `Drive Type`
5. `Steering Type`
6. `Front Brake Type`
7. `Rear Brake Type`
8. `Tyre Type`


In [39]:
# Make a list of such columns which seems to have a higher than expected cardinality
columns_unexpected_cardinality = [
    'Value Configuration',
    'Turbo Charger',
    'Gear Box',
    'Drive Type',
    'Steering Type',
    'Front Brake Type',
    'Rear Brake Type',
    'Tyre Type',
]

---

##### `Value Configuration`

In [40]:
cars_df[columns_unexpected_cardinality[0]].value_counts()

dohc                    23502
nan                      7831
sohc                     6200
undefined                 121
idsi                       66
dohc with vis              31
dohc with vgt              20
16 modules 48 cells        11
16-valve dohc layout        9
dohc with tis               9
vtec                        6
mpfi                        4
ohv / pushrod               3
Name: Value Configuration, dtype: int64

We have identified the following issues:
1. As we can see, the majority of it is `dohc`. There are a couple of variants of `dohc` like `dohc vis` but they are so few that it would be practical to replace all the columns containing any variant `dohc` to simply `dohc`
2. There are some `undefined` values which should be `NaN`
3. `mpfi` is a fuel injection system, so replace it with `NaN`
4. `vtec` engines can be `dohc` and `sohc` as well (`vtec` is an engine type developed by Honda in the 1960s). It would be best to replace this type by `NaN`, postpone handling it until null values handling (and there are only `6` rows, so it should be fine)

In [41]:
# Replace all the variants of `dohc` to simply `dohc`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vis', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vgt', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('16-valve dohc layout', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with tis', 'dohc')

# Replace `undefined`, `mpfi`, `vtec` with `NaN`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('undefined', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('mpfi', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('vtec', 'nan')

cars_df['Value Configuration'].value_counts()

dohc                   23571
nan                     7962
sohc                    6200
idsi                      66
16 modules 48 cells       11
ohv / pushrod              3
Name: Value Configuration, dtype: int64

---

##### `Turbo Charger`

In [42]:
cars_df[columns_unexpected_cardinality[1]].value_counts()

no       20247
yes      15262
nan       2176
twin       125
turbo        3
Name: Turbo Charger, dtype: int64

Well, cleaning this column should be quite straightforward.
1. `twin` and `turbo` should be replaced by `yes`

In [43]:
# Replace `twin` and `turbo` with `yes`
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('twin', 'yes')
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('turbo', 'yes')

cars_df['Turbo Charger'].value_counts()

no     20247
yes    15390
nan     2176
Name: Turbo Charger, dtype: int64

--- 

##### `Gear Box`

In [44]:
cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed                  22840
6 speed                   5876
5-speed                   1457
7 speed                   1256
6-speed                   1199
                         ...  
six speed manual             1
7g-dct                       1
amg speedshift dct 8g        1
amg 7-speed dct              1
ecvt                         1
Name: Gear Box, Length: 106, dtype: int64

Here the cardinality is high. And as is evident, several categories can be lumped together into a single one. Let's define a mapping to clearly indeicate which data should be lumped together.

In [45]:
gear_box_mapping = {}
gear_box_mapping['1 speed'] = [
	'single speed', 
	'single speed automatic', 
	'single speed reduction gear', 
	'single-speed transmission', 
]
gear_box_mapping['4 speed'] = [
	'4 speed', 
	'4-speed', 
]
gear_box_mapping['5 speed'] = [
	'5', 
	'5 - speed', 
	'5 gears', 
	'5 manual', 
	'5 speed', 
	'5 speed at+ paddle shifters', 
	'5 speed cvt', 
	'5 speed forward, 1 reverse', 
	'5 speed manual', 
	'5 speed manual transmission', 
	'5 speed+1(r)', 
	'5 speed,5 forward, 1 reverse', 
	'5-speed', 
	'5-speed`', 
	'five speed', 
	'five speed manual', 
	'five speed manual transmission', 
	'five speed manual transmission gearbox', 
]
gear_box_mapping['6 speed'] = [
	'6', 
	'6 speed', 
	'6 speed at', 
	'6 speed automatic', 
	'6 speed geartronic', 
	'6 speed imt', 
	'6 speed ivt', 
	'6 speed mt', 
	'6 speed with sequential shift', 
	'6-speed', 
	'6-speed at', 
	'6-speed automatic', 
	'6-speed autoshift', 
	'6-speed cvt', 
	'6-speed dct', 
	'6-speed imt', 
	'6-speed ivt', 
	'6-speed`', 
	'six speed  gearbox', 
	'six speed automatic gearbox', 
	'six speed automatic transmission', 
	'six speed geartronic, six speed automati', 
	'six speed manual', 
	'six speed manual transmission', 
	'six speed manual with paddle shifter', 
]
gear_box_mapping['7 speed'] = [
	'7 speed', 
	'7 speed 7g-dct', 
	'7 speed 9g-tronic automatic', 
	'7 speed cvt', 
	'7 speed dct', 
	'7 speed dsg', 
	'7 speed dual clutch transmission', 
	'7 speed s tronic', 
	'7-speed', 
	'7-speed dct', 
	'7-speed dsg', 
	'7-speed pdk', 
	'7-speed s tronic', 
	'7-speed s-tronic', 
	'7-speed steptronic', 
	'7-speed stronic', 
	'7g dct 7-speed dual clutch transmission', 
	'7g-dct', 
	'7g-tronic automatic transmission',
	'amg 7-speed dct',	
	'mercedes benz 7 speed automatic',
]
gear_box_mapping['8 speed'] = [
	'8', 
	'8 speed', 
	'8 speed cvt', 
	'8 speed multitronic', 
	'8 speed sport', 
	'8 speed tip tronic s', 
	'8 speed tiptronic', 
	'8-speed', 
	'8-speed automatic', 
	'8-speed automatic transmission', 
	'8-speed dct', 
	'8-speed steptronic', 
	'8-speed steptronic sport automatic transmission', 
	'8-speed tiptronic', 
	'8speed',
	'amg speedshift dct 8g', 
]
gear_box_mapping['9 speed'] = [
	'9 -speed', 
	'9 speed', 
	'9 speed tronic', 
	'9-speed', 
	'9-speed automatic', 
	'9g tronic', 
	'9g-tronic', 
	'9g-tronic automatic', 
	'amg speedshift 9g tct automatic',
]
gear_box_mapping['10 speed'] = [
	'10 speed', 
]
gear_box_mapping['cvt'] = [
'cvt', 
'e-cvt', 
'ecvt', 
]
gear_box_mapping['direct drive'] = [
'direct drive', 
]
gear_box_mapping['fully automatic'] = [
	'automatic transmission', 
	'fully automatic',
]
gear_box_mapping['nan'] = [
'nan',
'ags', 
'imt', 
'ivt', 
]

mapping_dict = {v: k for k, lst in gear_box_mapping.items() for v in lst}
cars_df['Gear Box'] = cars_df['Gear Box'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed            25203
6 speed             7459
7 speed             1842
8 speed             1150
4 speed              760
cvt                  474
nan                  471
9 speed              399
fully automatic       20
1 speed               15
direct drive          11
10 speed               9
Name: Gear Box, dtype: int64

--- 

##### `Drive Type`

In [46]:
cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd                                         27456
nan                                          4496
rwd                                          2248
awd                                          1082
2wd                                           648
4wd                                           570
2 wd                                          369
4x2                                           297
4x4                                           229
front wheel drive                             176
two wheel drive                                98
all wheel drive                                32
rear wheel drive with esp                      29
two whhel drive                                26
permanent all-wheel drive quattro              21
rwd(with mtt)                                  14
rear-wheel drive with esp                       7
4 wd                                            7
all-wheel drive with electronic traction        5
four whell drive                                2


We will once again take the same approach and define a mapping since many categories are redundant here.

In [47]:
drive_type_mapping = {}
drive_type_mapping['fwd'] = ['fwd', 'front wheel drive']
drive_type_mapping['2wd'] = ['2wd', 'two wheel drive', '2 wd', 'two whhel drive']
drive_type_mapping['rwd'] = ['rwd', 'rear wheel drive with esp', 'rear-wheel drive with esp', 'rwd(with mtt)']
drive_type_mapping['awd'] = ['awd', 'all wheel drive', 'all-wheel drive with electronic traction', 'permanent all-wheel drive quattro']
drive_type_mapping['4wd'] = ['4wd', '4 wd', '4x4', 'four whell drive']
drive_type_mapping['nan'] = ['nan', '3']

mapping_dict = {v: k for k, lst in drive_type_mapping.items() for v in lst}
cars_df['Drive Type'] = cars_df['Drive Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd    27632
nan     4497
rwd     2298
2wd     1141
awd     1140
4wd      808
4x2      297
Name: Drive Type, dtype: int64

---

##### `Steering Type`

In [48]:
# Steering Type
cars_df[columns_unexpected_cardinality[4]].value_counts()

power         31920
electric       3895
nan             808
manual          652
electronic      344
electrical      138
epas             49
hydraulic         5
mt                1
motor             1
Name: Steering Type, dtype: int64

There are 2 kinds of steering types (broadly) - 
1. Power (electric)
2. Manual (hydraulic, etc.)

In [49]:
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electrical', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electric', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electronic', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('epas', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('mt', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('motor', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')

cars_df[columns_unexpected_cardinality[4]].value_counts()

power     36348
nan         808
manual      657
Name: Steering Type, dtype: int64

---

##### `Front Brake Type` and `Rear Brake Type`

In [50]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc                                        22642
ventilated disc                             13178
ventilated discs                              400
solid disc                                    398
nan                                           327
disc & caliper type                           296
disk                                          205
ventilated disk                                78
drum                                           67
multilateral disc                              42
vantilated disc                                39
disc, 236 mm                                   22
ventlated disc                                 17
vacuum assisted hydraulic dual circuit w       12
ventillated disc                               11
disc & drum                                    10
disc,internally ventilated                      9
electric parking brake                          8
264mm ventilated discs                          8
caliper ventilated disc                         6


In [51]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum                                        29260
disc                                         5272
ventilated disc                               757
solid disc                                    612
nan                                           326
self-adjusting drum                           322
discs                                         301
disc & caliper type                           296
ventilated discs                              178
leading-trailing drum                         134
leading & trailing drum                        75
ventilated drum                                46
drums                                          40
self adjusting drum                            33
disc & drum                                    32
self adjusting drums                           31
drums 180 mm                                   22
drum in discs                                  17
vacuum assisted hydraulic dual circuit w       12
262mm disc & drum combination                   8


In [52]:
brake_type_mapping = {}
brake_type_mapping['disc'] = [
	'disc',
	'260mm discs',
	'disc brakes',
	'disc, 236 mm',
	'discs',
	'disk',
	'multilateral disc',
	'solid disc',
	'electric parking brake',
	'abs',
]
brake_type_mapping['ventilated disc'] = [
	'264mm ventilated discs',
	'booster assisted ventilated disc',
	'caliper ventilated disc',
	'disc brakes with inner cooling',
	'disc,internally ventilated',
	'vantilated disc',
	'ventilated & grooved steel discs',
	'ventilated disc',
	'ventilated disc with twin pot caliper',
	'ventilated discs',
	'ventilated disk',
	'ventillated disc',
	'ventillated discs',
	'ventlated disc',
	'ventilated drum in discs',
	'ventialte disc',
	'ventialted disc',
]
brake_type_mapping['carbon ceramic'] = [
	'carbon ceramic brakes',
	'carbon ceramic brakes.',
]
brake_type_mapping['disc & drum'] = [
	'disc & drum',
	'228.6 mm dia, drums on rear wheels',
	'262mm disc & drum combination',
	'drum in disc',
	'drum in discs',
]
brake_type_mapping['drum'] = [
	'drum',
	'203mm drums',
	'drum`',
	'drums',
	'drums 180 mm',
	'booster assisted drum',
	'drum brakes',
	'leading & trailing drum',
	'leading-trailing drum',
	'self adjusting drum',
	'self adjusting drums',
	'self-adjusting drum',
	'single piston sliding fist',
	'ventilated drum',
	'tandem master cylinder with servo assist',

]
brake_type_mapping['caliper'] = [
	'six piston claipers',
	'twin piston sliding fist caliper',
	'vacuum assisted hydraulic dual circuit w',
	'four piston calipers',
	'disc & caliper type',
]

In [53]:
mapping_dict = {v: k for k, lst in brake_type_mapping.items() for v in lst}
cars_df['Front Brake Type'] = cars_df['Front Brake Type'].replace(mapping_dict)
cars_df['Rear Brake Type'] = cars_df['Rear Brake Type'].replace(mapping_dict)

In [54]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc               23334
ventilated disc    13759
nan                  327
caliper              312
drum                  70
disc & drum           10
carbon ceramic         1
Name: Front Brake Type, dtype: int64

In [55]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum               29973
disc                6195
ventilated disc      944
nan                  326
caliper              310
disc & drum           64
carbon ceramic         1
Name: Rear Brake Type, dtype: int64

---

##### `Tyre Type`

In [56]:
cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless,radial               21770
tubeless                       5723
tubeless, radial               4994
tubeless tyres                 2408
radial                          606
radial, tubeless                389
tubeless radial tyres           301
radial, tubless                 296
nan                             257
tubeless tyres, radial          224
tubeless,radials                212
radial,tubeless                 158
tubless, radial                  92
runflat                          69
tubeless tyre                    50
tubeless, runflat                48
run-flat                         47
runflat tyres                    41
radial tubeless                  33
tubeless,runflat                 31
tubeless radial                  17
runflat tyre                     16
tubeless, radials                 7
radial with tube                  4
tubeless. runflat                 3
tubeless radial tyrees            3
tubeless tyres mud terrain        3
radial tyres                

In [57]:
tyre_type_mapping = {}
tyre_type_mapping['tubeless'] = [
	'tubeless tyres',
	'tubeless',
	'tubeless tyres mud terrain',
	'tubeless tyre',
]
tyre_type_mapping['tubeless radial'] = [
	'tubeless, radial',
	'tubeless,radial',
	'tubeless tyres, radial',
	'tubeless radial tyres',
	'radial, tubeless',
	'radial',
	'tubless, radial',
	'radial tubeless',
	'tubeless radial',
	'tubeless,radials',
	'tubeless radials',
	'radial,tubeless',
	'tubeless radial tyre',
	'radial, tubless',
	'tubless radial tyrees',
	'tubeless , radial',
	'tubeless, radials',
	'radial tyres',
]
tyre_type_mapping['runflat'] = [
	'runflat tyres',
	'runflat',
	'tubeless,runflat',
	'run-flat',
	'runflat tyre',
	'tubeless, runflat',
	'tubeless. runflat',
	'tubeless.runflat',
	'tubeless radial tyrees',
]
tyre_type_mapping['tube'] = [
	'radial with tube',
]

In [58]:
mapping_dict = {v: k for k, lst in tyre_type_mapping.items() for v in lst}
cars_df['Tyre Type'] = cars_df['Tyre Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless radial    29108
tubeless            8184
runflat              260
nan                  257
tube                   4
Name: Tyre Type, dtype: int64

---

##### `Fuel Injection System`

Although we expect a high cardinality from this feature, it's still worth a look if we could reduce the number of catgories by losing some of the variance.

(This step is optional and can be skipped. Also the mapping that is done is approximate and to the best of my knowledge, your mapping could differ)

In [59]:
cars_df['Fuel Suppy System'].value_counts()

mpfi                                      13443
crdi                                       9165
nan                                        5502
direct injection                           3831
pgm-fi                                      882
                                          ...  
intake port(multi-point)                      1
mfi                                           1
distributor-type diesel fuel injection        1
tfsi                                          1
pgm-fi (programmed fuel inject                1
Name: Fuel Suppy System, Length: 71, dtype: int64

In [60]:
fuel_injection_mapping = {
    "Gasoline Port Injection": [
    	"intelligent-gas port injection", 
    	"i-gpi",
    	"dohc",
    	"pfi"
	],
    "Multi-Point Fuel Injection": [
    	"mpfi", 
    	"multi-point injection", 
    	"mpfi+lpg", 
    	"mpfi+cng", 
    	"multipoint injection", 
    	"smpi", 
    	"mpi",
    	"multi point fuel injection",
    	"dpfi",
    	"mfi",
    	"multi point injection",
    	"msds",
    	"cng"
    ],
    "Electronic Fuel Injection": [
	    "efi(electronic fuel injection)", 
	    "efi", 
	    "efi (electronic fuel injection)", 
	    "efic", 
	    "electronic fuel injection", 
	    "electronically controlled injection", 
	    "electronic injection system", 
	    "sefi",
	    "egis",
	    "efi (electronic fuel injection",
	    "efi",
        "efi -electronic fuel injection",
    ],
    "Direct Injection": [
    	"direct injection", 
    	"direct injectio", 
    	"direct fuel injection",
    	"direct engine",
    ],
    "Common Rail Injection": [
    	"crdi", 
    	"common rail", 
    	"common rail injection", 
    	"common rail direct injection", 
    	"common rail direct injection (dci)", 
    	"common-rail type", 
    	"advanced common rail", 
    	"common rail system", 
    	"common rail diesel", 
    	"pgm-fi (programmed fuel injection)", 
    	"pgm-fi (programmed fuel inje", 
    	"pgm - fi", 
    	"pgm-fi", 
    	"pgm-fi (programmed fuel inject",
    	"direct injection common rail",
    	"cdi"
    ],
    "Distributor-Type Fuel Injection": [
    	"dedst", 
    	"distribution type fuel injection", 
    	"distributor-type diesel fuel injection",
    ],
    "Indirect Injection": [
    	"indirect injection",
    	"idi"
    ],
    "Gasoline Direct Injection": [
    	"gdi", 
    	"gasoline direct injection",
    	"tfsi",
    	"tsi",
    	"tgdi"
    ],
    "Turbo Intercooled Diesel": [
    	"tcdi", 
    	"turbo intercooled diesel",
    	"tdci"
    ],
    "Intake Port Injection": [
    	"intake port(multi-point)"
    ],
    "Diesel Direct Injection": [
    	"ddi", 
    	"ddis"
    ],
    "Variable Valve Timing Injection": [
    	"dual vvt-i", 
    	"vvt-ie", 
    	"ti-vct"
    ],
    "Three-Phase AC Induction Motors": [
    	"3 phase ac induction motors"
    ],
    "Electric": [
    	"electric", 
    	"isg"
    ],
}

mapping_dict = {v: k for k, lst in fuel_injection_mapping.items() for v in lst}
cars_df['Fuel Suppy System'] = cars_df['Fuel Suppy System'].replace(mapping_dict)

cars_df['Fuel Suppy System'].value_counts()

Multi-Point Fuel Injection         14055
Common Rail Injection              11822
nan                                 5502
Direct Injection                    3837
Electronic Fuel Injection           1593
Gasoline Direct Injection            653
Gasoline Port Injection              100
Diesel Direct Injection               99
Turbo Intercooled Diesel              89
Indirect Injection                    21
Variable Valve Timing Injection       15
Three-Phase AC Induction Motors       11
Distributor-Type Fuel Injection        8
Electric                               7
Intake Port Injection                  1
Name: Fuel Suppy System, dtype: int64

##### With this column, all of the features suspected of high cardinality are handled. We can move on to the other things that were pointed out earlier in the code.

---

#### Converting features to numerical (and/or boolean)

The features we wish to convert are the following:
1. `pu` to float
2. `Max Power` to 2 separate columns: `Max Power Delivered` and `Max Power At`
3. ~~`No of Cylinder` to int (from float)~~
4. ~~`Values per Cylinder` to int (from float)~~
5. `BoreX Stroke` to 2 separate columns: `Bore` and `Stroke`
6. `Turbo Charger` to boolean
7. `Super Charger` to boolean
8. `Length` to int 
9. `Width` to int
10. `Height` to int
11. `Wheel Base` to int
12. `Front Tread` to int
13. `Rear Tread` to int
14. `Kerb Weight` to int
15. `Gross Weight` to int
16. ~~`Seating Capacity` to int (from float)~~
17. `Turning Radius` to float
18. `Top Speed` to int
19. `Acceleration` to float
20. ~~`No Door Numbers` to int (from float)~~
21. `Cargo Volumn` to int
22. `Compression Ratio` to float
23. `Alloy Wheel Size` to float
24. `Ground Clearance Unladen` to int

In [61]:
# Make a copy of the data, cars_df will now serve as a checkpoint
cars_df2 = cars_df.copy()

# Sanity check
cars_df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,180-liters,used,uttar pradesh,silver,first,nan,NaN,nan,nan
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,5.000,235-litres,used,maharashtra,grey,first,Gasoline Port Injection,11.0:1,nan,nan
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,4.000,400-litres,used,delhi,silver,second,nan,NaN,nan,nan
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,"1,15,893",0,[{'img': ''}],0,False,...,4.000,nan,used,delhi,silver,second,Multi-Point Fuel Injection,NaN,13,nan
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,5.000,nan,used,maharashtra,white,first,nan,12.0+-.03,nan,nan


In [62]:
# A utility function to get a number from a string
def converst_to_number(x, conv: str = 'float'):
    x = str(x)
    new_str = ''
    is_dec = True
    for a in x:
        if 48 <= ord(a) <= 57:
            new_str += a
            continue
        elif a == ',' or a == '_':
            continue
        elif a == '.' and is_dec:
            new_str += a
            is_dec = False
        else:
            break
    
    if new_str == '':
        return None
    
    if conv == 'int':
        return int(new_str)
    
    return float(new_str)

def get_begin_number(x):
    return converst_to_number(x, 'int')

def get_begin_float(x):
    return converst_to_number(x, 'float')

# def get_begin_number(x):
#     x = str(x)
#     reg_match = re.search(r'\d+', x)
#     if reg_match is None:
#         return None
#     return int(reg_match.group())

# def get_begin_float(x):
#     x = str(x)
#     reg_match = re.search(r'\d+\.?\d+', x)
#     if reg_match is None:
#         return None
#     return float(reg_match.group())

---

#### `pu`

This is our target column (the price of our used cars). This should be a float or int.

In [63]:
cars_df2['pu'].sample(5)

usedCarSkuId
36d4d2a0-43c2-4e70-8198-519651ad8d0f    5,20,000
f7649acf-95b4-4409-9354-fc0a8ec9bf42    2,85,000
99fc68d3-c7cc-4344-a165-6ae54194078f    4,26,000
a06bd352-3ba2-4edd-a199-eb46e93f291c    5,75,000
43662859-0d13-456c-8aeb-193adc2320f2      75,075
Name: pu, dtype: object

In [64]:
# We simply need to replace the ',' to with '' and convert
cars_df2['pu'] = cars_df2['pu'].str.replace(',', '').astype(float)

cars_df2['pu'].sample(5)

usedCarSkuId
c3f389c2-33ab-4c27-a191-122447aabada   350000.000
bc3243f4-23f1-4ca7-92d9-51732870228c   315000.000
320873ff-81c0-4844-8df0-91d15feb333b   350000.000
5571b05e-8d1e-436e-aa72-9f28f3836f63   270000.000
c7a4b4e9-4b61-40c6-a5dc-c5a3050aa202   225000.000
Name: pu, dtype: float64

---

#### `Max Power` and `Max Torque`

We will be separating this column into 2 columns: `Max Power Delivered` and `Max Power At` (containing the rpm at which the max power is reach)
Same with `Max Torque`

In [65]:
cars_df2[['Max Power', 'Max Torque']].sample(5)

,Max Power,Max Torque
usedCarSkuId,,
e26c1e08-bd37-4f84-84f4-e3a3fa51d005,74bhp@4000rpm,190nm@2000rpm
4f9b5728-cefa-46ff-adba-0a0772a781e7,235bhp@4250rpm,480nm@1750rpm
c6ed20ce-60e2-4a55-9cf4-fcf490ddc090,108.6bhp@4000rpm,250nm@1500-2500rpm
453f56c4-5b33-4df1-a7c1-afd20e351857,85.80bhp@6000rpm,114nm@4000rpm
1aeb09cc-3f23-49bb-b9d5-364e270933f1,86.7bhp@6000rpm,109nm@4500rpm


In [66]:
c = cars_df2.copy()
cars_df2['Max Power Delivered'] = cars_df2['Max Power'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Power At'] = cars_df2['Max Power'].str.split('@').str[1].apply(get_begin_float).astype(float)


def get_rpm_average(x):
    x = str(x)
    if '-' in x:
        p1 = get_begin_float(x.split('-')[0])
        p2 = get_begin_float(x.split('-')[1])
        if p1 is None:
            return p2
        if p2 is None:
            return p1
        
        return (p1 + p2)/2
    else:
        return get_begin_float(x)

cars_df2['Max Torque Delivered'] = cars_df2['Max Torque'].str.split('@').str[0].apply(get_begin_float).astype(float)
cars_df2['Max Torque At'] = cars_df2['Max Torque'].str.split('@').str[1].apply(get_rpm_average).astype(float)

In [67]:
cars_df2[['Max Power Delivered', 'Max Power At', 'Max Torque Delivered', 'Max Torque At']].sample(5)

,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,
fe4d80b3-7aca-45c9-a07a-90d92b489dfe,108.500,5000.000,170.000,2875.000
a7e2742a-2603-429d-b88d-cb6a09d0b4e4,88.730,4000.000,219.700,2125.000
41f1fdf9-8e84-4869-aa97-c202af54aeae,98.600,3600.000,200.000,1750.000
252852cd-a9f2-4c09-aa2b-dc8f32b0bc80,NaN,NaN,NaN,NaN
8d368493-f490-4d59-9f5c-d1e796ee4a8f,67.100,6200.000,90.000,3500.000


Now we can safely drop `Max Power` from our dataframe

In [68]:
cars_df2.drop(columns=['Max Power'], inplace=True, axis=1)
cars_df2.drop(columns=['Max Torque'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,silver,first,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,grey,first,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,silver,second,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,"1,15,893",0,[{'img': ''}],0,False,...,silver,second,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,white,first,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000


---

#### `BoreX Stroke`

We will make 2 new columns `Bore` and `Stroke` and remove the original column.


In [69]:
cars_df2['BoreX Stroke'].sample(10)

usedCarSkuId
5f81e4ab-c86a-4cd4-b514-05f07f3d3558               nan
fd3d53e8-e5bf-4fdc-ad59-678e443a307f        69 x 72 mm
aebafd28-36f1-4aa0-aa3c-c6a6ace6dfa2               nan
88fae56a-eff9-4b96-bc07-dc1b963487ea               nan
72952a90-e269-4f08-b036-a2e3362d61f9        75 x 87 mm
051011d1-71f3-4d71-888c-4da00e084f58    73.0 x 89.4 mm
29e84006-1fca-49cf-82b9-a822362a7564      69.6 x 82 mm
f01980bc-2789-4992-9ec3-0e2cc10f80bd        66 x 77 mm
41925705-f840-4318-a8a5-fa631d4dde6b    73.0 x 89.4 mm
f68c0562-db5b-440e-82f5-76351f4a1821               nan
Name: BoreX Stroke, dtype: object

In [70]:
cars_df2['Bore'] = cars_df2['BoreX Stroke'].str.split('x').str[0].apply(get_begin_float).astype(float)
cars_df2['Stroke'] = cars_df2['BoreX Stroke'].str.split('x').str[1].apply(get_begin_float).astype(float)

In [71]:
cars_df2[['Bore', 'Stroke']].sample(10)

,Bore,Stroke
usedCarSkuId,,
efe5420e-3aba-46d5-8874-5f113ef3b6a8,NaN,NaN
03b6db47-2921-4d5e-8944-78047a9a94d3,NaN,NaN
b0b2d0e8-bfe1-4516-bf30-ca24d6b057b4,NaN,NaN
060ef678-025c-4269-a33b-d6e9f866a509,NaN,NaN
09074205-31d4-41e2-87f2-30e588c05b17,NaN,NaN
ec927074-c486-43ba-9f16-e7ac4b7c4c96,NaN,NaN
51df3ba1-8bad-4a3a-ab9f-cabdda85dc1c,NaN,NaN
dac5ed49-3978-4f9c-905f-cb0241952f5c,NaN,NaN
d02e1534-9320-4a92-9a59-a0974fb4415e,NaN,NaN


In [72]:
cars_df2.drop(columns=['BoreX Stroke'], inplace=True, axis=1)
cars_df2.head()

,loc,myear,bt,tt,ft,km,ip,images,imgCount,threesixty,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,"69,162",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,nan,nan,58.160,6200.000,77.000,3500.000,69.000,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,"45,864",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,Gasoline Port Injection,11.0:1,nan,nan,58.200,6000.000,78.000,3500.000,73.000,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,"81,506",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,nan,nan,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,"1,15,893",0,[{'img': ''}],0,False,...,Multi-Point Fuel Injection,NaN,13,nan,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,"18,900",0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,nan,12.0+-.03,nan,nan,86.630,5500.000,121.500,4200.000,NaN,NaN


---

#### `Turbo Charger` and `Super Charger`

Convert them to boolean

In [73]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
b9f97f8c-29f4-455b-a631-af71c0774c2e,no,no
7a62abbf-b8c9-410d-a68b-11b1465b431f,no,no
49aed440-51db-45f0-9c9b-b16e636c3da3,no,no
90977d2c-849d-40fd-a5cc-0298e050a60d,no,no
9daaf6d9-559c-4bef-afcc-ed9dd7124ee3,no,no


In [74]:
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Turbo Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('yes', True)
cars_df2['Super Charger'] = cars_df2['Turbo Charger'].replace('no', False).astype(bool)

In [75]:
cars_df2[['Turbo Charger', 'Super Charger']].sample(5)

,Turbo Charger,Super Charger
usedCarSkuId,,
ac134c42-b7f3-417a-ba1d-b97849132781,True,True
acaedb1a-e323-4fb7-a659-a7437abd9bef,True,True
d8457dc9-f237-49dc-a15a-f343fc6abea8,False,False
b475145f-d05b-4c17-8705-5ed3eb02b05a,True,True
58a266f5-c9a1-4b12-a217-5aee8d6a8565,False,False


---

#### `Length`, `Width`, `Height`, `Wheel Base`

Convert these columns to int

In [76]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(10)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
bf54bba3-95ce-460e-94ee-165b7a3c0fcf,4000mm,1642mm,1522mm,2465mm
02e1ba40-019f-4534-8ec7-1f0fcd98c35b,4315mm,1822mm,1695mm,2673mm
ac8710c9-2f75-4d40-9add-bc27a18de354,3985mm,1734mm,1505mm,2570mm
2f5f408a-2e26-4c44-b34b-4052951a5c3b,3495mm,1475mm,1460mm,2360mm
7b48674a-eeb0-4267-85c5-e6fe5a61378b,4370mm,1700mm,1475mm,2570mm
551d7e1d-d293-4df2-972c-be3771fb6453,3995mm,1790mm,1640mm,2500mm
bbe3e475-a7e0-4ba1-ae50-ecb327796177,4720mm,1835mm,1760mm,2750mm
d052d312-7732-4a0e-8121-a7fde26e8702,3545mm,1515mm,1475mm,2360mm
88dcafbc-37f5-49a8-a349-1948ec3f8b0a,3445mm,1515mm,1475mm,2360mm


In [77]:
for col in ['Length', 'Width', 'Height', 'Wheel Base']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [78]:
cars_df2[['Length', 'Width', 'Height', 'Wheel Base']].sample(5)

,Length,Width,Height,Wheel Base
usedCarSkuId,,,,
acbc57eb-00a2-441a-a879-c964528350a9,3970.000,1682.000,1453.000,2456.000
2622f334-9d24-44c2-a29e-1d16d6077071,4939.000,2110.000,1470.000,2930.000
c9b64dcd-0dc6-4798-8aa4-331c7e9b5a00,4270.000,1780.000,1665.000,2590.000
0055cb61-1f99-4cec-8d38-3edd65d9382b,3765.000,1660.000,1520.000,2425.000
9a26b69c-77a2-44c5-9c07-43b81de4ad7f,4270.000,1780.000,1630.000,2590.000


---

#### `Front Tread` and `Rear Tread`

In [79]:
cars_df2[['Front Tread', 'Rear Tread']].sample(10)

,Front Tread,Rear Tread
usedCarSkuId,,
13eebdea-94c6-41f0-a59c-64dcee216733,nan,nan
a9dc435a-e712-49a6-b203-ca8c42c2b66a,1295mm,1290mm
5b59952e-209f-4d2a-b0c3-0f9e93bdf04b,nan,nan
4866b787-6699-4e7f-b53f-4f7b7e2a30c2,nan,nan
11318d9f-d768-4fc1-9c15-03cade33aedc,1295mm,1290mm
038a76c3-77c2-43c1-8548-1381fa3c71d3,1571mm,1575mm
39f7bf7b-6b04-4a6d-9945-c04e6526d973,1515mm,1525mm
2809af63-3c50-4092-b737-9461e1e65ad6,nan,nan
81af046e-d116-49a3-9858-ea9e666f436e,nan,nan


In [80]:
for col in ['Front Tread', 'Rear Tread']:
    cars_df2[col] = cars_df2[col].apply(get_begin_float).astype(float)

In [81]:
cars_df2[['Front Tread', 'Rear Tread']].sample(5)

,Front Tread,Rear Tread
usedCarSkuId,,
f7f13df7-6c51-4ddc-813b-b2f80e802cdb,1470.000,1475.000
a1f40a82-d841-4136-a148-e480b32259f0,1485.000,1495.000
ad73cd8d-8690-4e5d-b8d7-9342c581661b,1485.000,1495.000
c0654ca9-cccc-4a0e-9d6a-cbd48741bcba,NaN,NaN
cceaac36-c4ff-46fc-9e43-8c1137f4cde1,1485.000,1495.000


---

#### `Kerb Weight` and `Gross Weight` 

Convert weights to numerical features

In [82]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
9e66f83e-aecd-4d98-aa0d-613b59b21380,1238kg,1770kg
0b00291a-43e8-40a9-9042-0863f4f0f839,nan,nan
1afd66ea-e68b-49e5-8358-ae9b211a3652,1065kg,nan
bc0bb3e7-1f3b-46b5-9738-1743827a3762,nan,nan
0df96a5a-8c3f-4a18-b8ab-ebe3ed9e8b6d,nan,2225kg
05d822a5-5005-459f-9573-c1dbea87620b,1066kg,nan
e9e1e0e7-b9fb-4370-b072-b06d4029c5ff,nan,nan
6f33ae2d-747b-4abe-9904-6688b9090521,1206kg,1760kg
7528bb83-11cf-4d2f-abb7-48d63fd87704,nan,nan


In [83]:
for col in ['Kerb Weight', 'Gross Weight']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [84]:
cars_df2[['Kerb Weight', 'Gross Weight']].sample(10)

,Kerb Weight,Gross Weight
usedCarSkuId,,
d1ebd8cb-893d-42b6-a3e9-a141f8bc9b08,1824.000,2265.000
93cbfaa9-9ef9-4f3b-8fbf-e5513a455377,720.000,1185.000
71990da8-5f12-4489-b74e-9eb4d42ff4f4,1065.000,NaN
e690c74c-5e82-42a4-91d6-e949a97185ce,1198.000,NaN
23aeae0c-99dd-4af7-9969-83d8f438c49b,NaN,NaN
5e613143-86a2-4618-bc61-7229312e7fd9,960.000,1405.000
5df4a9c6-a4fa-42be-a996-076b54dc4959,1230.000,1670.000
4b6aec56-0ec2-4729-a485-c16f2a25ddf2,1180.000,1680.000
2ee70626-96c6-4a8c-8c57-07a613c682eb,1170.000,NaN


---

#### `Turning Radius`,

In [85]:
cars_df2['Turning Radius'].sample(10)

usedCarSkuId
d4ba3369-f970-471e-98f4-d61d1c5e8e71     4.8 metres
14fbd704-6a18-4ade-b94a-1b3e1f09f86c     5.3 metres
abf51948-5939-4fc0-a4c4-e52d3b94d71f            nan
495ca245-566b-4e6b-842d-35ebea37300b     5.3 metres
732097fb-05ab-421b-9422-8b51159aa1ad     5.3 metres
97d737bb-7ca5-46e8-8aa0-8cf09b20bb2e     5.3 metres
466a6200-bd57-4fdb-8e25-2c3f68459c7b    5.65 metres
423d2d42-861d-43cb-8534-23298d879562     4.7 metres
2399ec56-ffec-4729-a618-9461331e9f74     4.7 metres
56425b8a-7689-46af-b71c-049e70cf43d2            nan
Name: Turning Radius, dtype: object

In [86]:
cars_df2['Turning Radius'] = cars_df2['Turning Radius'].apply(get_begin_float).astype(float)

In [87]:
cars_df2['Turning Radius'].sample(5)

usedCarSkuId
9a1111ab-f0e6-42e9-bee7-514eef20ebc3   5.200
80a89d50-78b3-4dd5-812b-8de82f68e8e3   4.800
29f87464-926d-432b-8dd4-a187b7f1684c   4.700
1b7b4c3d-d186-4eee-9239-c6828792552c   4.600
67ca4489-e7fc-4142-a7e4-1b7f600dc5e8   5.300
Name: Turning Radius, dtype: float64

---

#### `Top Speed` and `Acceleration`

In [88]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
05448c08-ab41-4681-bda8-bade62cf9b1d,190 kmph,14.5 seconds
8c821f81-47a2-46b3-98b9-47ee70f3baf9,nan,nan
93b9d251-5bea-4079-b195-f87cb54dcbc4,180 kmph,12.36 seconds
60c02ce0-d274-4f56-afb3-281e8b1f0979,140 kmph,15 seconds
53197e9e-2328-40e6-b1de-7cea7d6b2bfa,206 kmph,9.6 sconds
9f8fc20c-2911-4431-a5e3-0ae64de69f79,152 kmph,18.6 seconds
50d27ab1-7db7-48c9-8561-567abf6b88ac,nan,nan
56c8ab92-b49d-420d-962c-3392f9a238ff,182 kmph,16 seconds
6d973cbf-7a6b-4df7-9e13-0873efef4ae3,nan,12.71s


In [89]:
cars_df2['Top Speed'] = cars_df2['Top Speed'].apply(get_begin_float).astype(float)
cars_df2['Acceleration'] = cars_df2['Acceleration'].apply(get_begin_float).astype(float)

In [90]:
cars_df2[['Top Speed', 'Acceleration']].sample(10)

,Top Speed,Acceleration
usedCarSkuId,,
7b43b57a-1489-4671-9958-539c36a14634,NaN,NaN
ddf907e7-512a-4e6c-a0a6-28498b297aac,205.000,7.900
2c26268f-8ef4-472c-bdf1-8b891d8a4a0d,NaN,NaN
c88afaa2-3d12-4e52-8dbe-8a8c1f31dce6,NaN,NaN
d4232e0a-e640-4e7b-ad7c-c054eeb45069,140.000,19.000
ee6ce523-2aa6-4852-96a1-86b9bc277377,154.190,13.250
ecadf904-0c13-40b3-bab3-8e1f7f7db469,NaN,NaN
b017bd6c-54f9-483e-bff9-a9394030b3a3,172.000,12.360
4226589d-0e5f-43ab-98bc-f3c52cb1bf06,155.800,13.900


---

#### `Cargo Volumn` and `Ground Clearance Unladen`

Convert these columns to int

In [91]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
f07e2d82-8b88-400c-b84b-f05557aea952,353-litres,nan
83bf5dc3-69a4-4a44-8658-d5dece1386b6,170-litres,nan
6eddec5d-d3cc-43d9-a426-60c544a1aa31,400-litres,nan
5fcc66e6-1f6c-41fa-9048-1b05844b79a8,nan,nan
b274bb3b-1092-4a47-bd83-3f8f853cf0a2,350,190mm
e09f47aa-d5ac-469e-9bdb-1df7808f8427,242-litres,nan
f5ad2135-147c-41d2-b1eb-10ba25d5add1,420-litres,nan
e9b7d07a-4238-455e-adaa-aafff2bed9c6,560 litres,nan
53ac4ca7-9f63-40bd-ac0d-6336e650272b,392,nan


In [92]:
for col in ['Cargo Volumn', 'Ground Clearance Unladen']:
    cars_df2[col] = cars_df2[col].apply(get_begin_number).astype(float)

In [93]:
cars_df2[['Cargo Volumn', 'Ground Clearance Unladen']].sample(10)

,Cargo Volumn,Ground Clearance Unladen
usedCarSkuId,,
d673e581-ae35-4994-82f7-0ab8cf00723c,232.000,NaN
a5d5f9e3-a4c3-48cb-8583-52f1ee8b0cf9,433.000,NaN
694989ce-5821-4767-bbea-1d96f971d979,354.000,NaN
283ea6b2-ff3c-4c27-b909-36eb0f2f41ef,510.000,NaN
68062a66-9993-42bb-a245-c7dbaf33ac94,400.000,NaN
cbd4f4e4-c3be-4613-b695-3ca870d523a3,280.000,NaN
e09adcc2-d6f2-4946-bf88-a9ad7884419e,NaN,NaN
b91ff817-1620-41d1-8e6b-d3751a2f8f93,405.000,NaN
f35caadf-0b65-4eb2-9f3c-1d7a27cb3620,510.000,NaN


---

#### `Compression Ratio` and `Alloy Wheel Size`

These columns should be converted to floats

In [94]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
63706ab7-23bb-4d93-860b-1847908de5ea,16,10.3:1
ab417162-2739-4e94-aabb-c04974f59918,16,NaN
a39a0481-ee90-49ad-bd35-d1ea687631be,18,NaN
64ab1674-2c9f-4b4d-ad77-afffb191497e,nan,NaN
443efd97-b2b7-4c4e-bd37-30de6894c17f,18,NaN
8ead7549-7ff9-494a-bafe-c4bf7bc26440,nan,NaN
f74e5c20-a2b7-4478-a2be-d89d7946b821,17,NaN
1f70930f-f25f-44c2-8164-14848f6e75f4,14,NaN
64dc6c23-b90b-4845-a13b-2fbd06bd2149,15,81 x 87.3 mm


In [95]:
for col in ['Alloy Wheel Size', 'Compression Ratio']:
    cars_df2[col] = cars_df2[col].apply(get_begin_float).astype(float)

In [96]:
cars_df2[['Alloy Wheel Size', 'Compression Ratio']].sample(10)

,Alloy Wheel Size,Compression Ratio
usedCarSkuId,,
7b12adf8-89f5-48b4-a8f3-9db46940397f,NaN,NaN
ef67af42-dd4f-4dac-9894-a0b0f2ee065d,NaN,NaN
b57d71d6-20a4-4e76-9929-2cc680840f00,16.000,9.010
d5ce8641-f29f-4bff-905d-5ad9adc3cfe0,17.000,NaN
dd99a377-51cd-4245-85f1-6f34f38b9a20,16.000,17.300
b2d17a6a-5e7a-4c69-a120-7327e147e23f,15.000,NaN
17a58eeb-5858-49c7-997a-d63c467c339d,NaN,NaN
dc2fbe92-f0f1-41db-820a-141f339a461d,14.000,9.000
57d24fba-ae96-47e0-b32c-2d686834db01,15.000,10.300


---
### `km`

The columns need to be converted to int64.

In [97]:
cars_df2['km'].sample(10)

usedCarSkuId
6d140421-c08e-426a-b900-e84677e286a1      33,243
996b7c79-9846-44f4-9624-b32c0ebce8df      18,710
bad5d41a-1976-49f9-8d04-12e5893ca11d      73,698
90284ea4-a16a-4ebe-8813-465885c406b4      31,000
edbc7b55-8cdf-405f-bd60-76671607ac0c    1,19,255
987b3f9f-2369-443e-9149-4591b5233cf9      12,100
2661df70-3fc4-4a7e-a319-aba0e8a49633      68,313
3b740011-d7b7-408a-9723-84edddec85e1       2,322
4e59b62e-907c-4768-8a4c-273f5b7a6f2b      45,000
c0e89987-72e4-44cd-8b3b-1eb72a523453      92,000
Name: km, dtype: object

In [98]:
cars_df2['km'] = cars_df2['km'].apply(get_begin_float).astype(float)

In [99]:
cars_df2['km'].sample(10)

usedCarSkuId
79187f95-021f-4900-bdaf-4f22b9306931   38324.000
73ddfcc5-f445-4575-aca6-848104c70015   33120.000
7c37191a-0b6b-48a1-9c49-7d6c0052d0e3   74970.000
a4933290-f39b-40b2-b486-0cb0b78cf2db   54955.000
acb59ce6-b6e3-4b6f-b8a9-b159e344ad88   27155.000
647a8600-314b-4bd8-a475-e718aa082d0f   17924.000
bc962855-6535-4a69-8d84-33c5686fe317   46000.000
9bfe650e-0473-4d30-807e-0dc05ac570ab   61353.000
37e75586-3bb1-4732-92d6-5fd56246ac01   10172.000
865bae8c-8860-4165-b480-b91a7178c0d3   31098.000
Name: km, dtype: float64

With this, we have converted all the columns to numerical that were noted.

In [100]:
cars_df2.dtypes.value_counts()

object     31
float64    27
int64       4
bool        3
dtype: int64

---

#### Fixing some column names

In [101]:
cars_df2 = cars_df2.rename(columns={
    'tt': 'transmission',
    'bt': 'body',
    'ft': 'fuel',
    'variantName': 'variant',
    'km': '',
    'pu': 'listed_price',
    'Values per Cylinder': 'Valves per Cylinder',
    'Value Configuration': 'Valve Configuration',
    'No Door Numbers': 'Doors',
    'Seating Capacity': 'Seats',
    'Cargo Volumn': 'Cargo Volume',
    'city_x': 'City',
})

In [102]:
cars_df2.head()

,loc,myear,body,transmission,fuel,,ip,images,imgCount,threesixty,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,69162.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,NaN,NaN,58.160,6200.000,77.000,3500.000,69.000,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,45864.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,Gasoline Port Injection,11.000,NaN,NaN,58.200,6000.000,78.000,3500.000,73.000,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,81506.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,...,nan,NaN,NaN,NaN,86.700,6000.000,109.000,4500.000,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,115893.000,0,[{'img': ''}],0,False,...,Multi-Point Fuel Injection,NaN,13.000,NaN,58.200,6200.000,77.000,3500.000,NaN,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,18900.000,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,...,nan,12.000,NaN,NaN,86.630,5500.000,121.500,4200.000,NaN,NaN


In [103]:
cars_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37813 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Data columns (total 65 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   loc                       37813 non-null  object 
 1   myear                     37813 non-null  int64  
 2   body                      37813 non-null  object 
 3   transmission              37813 non-null  object 
 4   fuel                      37813 non-null  object 
 5                             37813 non-null  float64
 6   ip                        37813 non-null  int64  
 7   images                    37813 non-null  object 
 8   imgCount                  37813 non-null  int64  
 9   threesixty                37813 non-null  bool   
 10  dvn                       37813 non-null  object 
 11  oem                       37813 non-null  object 
 12  model                     37813 non-null  object 
 13  

#### With this, we are ready to move on to the next steps:
1. EDA
2. Feature Engineering and selection
3. Model Building and Optimization

In [105]:
import datetime

fileName = f"../data/clean/cleaned_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
cars_df2.to_csv(fileName)